In [ ]:
# 2020-10-21 created by Akson

In [ ]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url = HOUSING_URL, housing_path = HOUSING_PATH):
    os.makedirs(housing_path, exist_ok = True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path = housing_path)
    housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
# Code2.1
# 加载数据

import pandas as pd

# housing = pd.read_csv('/content/drive/MyDrive/datasets/housing/housing.csv')
def load_housing_data(housing_path = HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
housing = load_housing_data()

In [ ]:
# Code2.2
# 显示头五行数据

housing.head()

In [ ]:
# Code2.3
# 显示info

housing.info()

In [ ]:
# Code2.4
# 检视'ocean_proximity'列

print(housing['ocean_proximity'].value_counts())
print(housing['ocean_proximity'])

In [ ]:
# Code2.5
# 试试.describe()

housing.describe()

In [ ]:
# Code2.6
# 绘制每种属性的直方图

%matplotlib inline
import matplotlib.pyplot as plt

housing.hist(bins = 50, figsize = (20, 15))
plt.show()

In [ ]:
# Code2.7
# 随机拆分数据集

from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size = 0.2, random_state = 42)

In [ ]:
# Code2.8
# 收入数据分层

import numpy as np

housing['income_cat'] = pd.cut(housing['median_income'], bins = [0., 1.5, 3.0, 4.5, 6.0, np.inf], labels = [1, 2, 3, 4, 5])
housing['income_cat'].hist()

In [ ]:
# Code2.9
# 分层抽样

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)

for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# strat_train_set
# strat_train_set['income_cat'].value_counts() / len(strat_train_set)

# 用完就丢，您可真是个带恶人
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis = 1, inplace = True)

In [ ]:
# Code2.10
# 详细探究数据

# 为了能随意操作数据而不影响原数据集，创建一个副本（得益于这个数据集较小，大数据及要慎重）
housing = strat_train_set.copy()

housing.plot(kind = 'scatter', x = 'longitude', y = 'latitude', alpha = 0.1)

In [ ]:
# Code2.11
# 再加上房价与人口因素

housing.plot(kind = 'scatter', x = 'longitude', y = 'latitude', alpha = 0.4, s = housing['population'] / 100, label = 'population', figsize = (10, 7), c = 'median_house_value', cmap = plt.get_cmap('jet'), colorbar = True)
plt.legend()

In [ ]:
# Code2.12
# 计算每对属性的相关系数，并提取出与房价有关的信息

corr_matrix = housing.corr()

corr_matrix['median_house_value'].sort_values(ascending = False)

In [ ]:
# Code2.13
# 使用pandas.scatter_matrix

from pandas.plotting import scatter_matrix

attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
scatter_matrix(housing[attributes], figsize = (12, 8))

# 也可以都输出试试
# scatter_matrix(housing[:], figsize = (12, 8))

In [ ]:
# Code2.14
# 看看感觉上与房价最相关的收入中位数

housing.plot(kind = 'scatter', x = 'median_income', y = 'median_house_value', alpha = 0.1)

In [ ]:
# Code2.15
# 根据存在的各种特征信息组合看看能不能导出其它更加有意义的特征

housing['rooms_per_household'] = housing['total_rooms'] / housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms'] / housing['total_rooms']
housing['population_per_household'] = housing['population'] / housing['households']

corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending = False)

In [ ]:
# Code2.16
# 处理缺失数据（pandas提供的方法，丢弃缺失的部分）

# 从数据集中去除作为标签的数据
housing = strat_train_set.drop('median_house_value', axis = 1)

housing.dropna(subset = ['total_bedrooms'])  # 丢弃缺失数据的部分

In [ ]:
# Code2.17
# 处理缺失数据（pandas提供的方法，丢弃整个属性）

# 从数据集中去除作为标签的数据
housing = strat_train_set.drop('median_house_value', axis = 1)

housing.drop('total_bedrooms', axis = 1)  # 丢弃整个‘total_bedrooms’属性

In [ ]:
# Code2.18
# 处理缺失数据（pandas提供的方法，用中位数填补）

# 从数据集中去除作为标签的数据
housing = strat_train_set.drop('median_house_value', axis = 1)

# 使用中位数填充（也可以是其它值）
median = housing['total_bedrooms'].median()
housing['total_bedrooms'].fillna(median, inplace = True)

In [ ]:
# Code2.19
# 处理缺失数据（使用sklearn提供的方法）

from sklearn.impute import SimpleImputer

# 从数据集中去除作为标签的数据
housing = strat_train_set.drop('median_house_value', axis = 1)

# 由于SimpleImputer只能操作数字，所以还要将数据中的非数字列去掉
housing_num = housing.drop('ocean_proximity', axis = 1)

# 定义转换器对象
imputer = SimpleImputer(strategy = 'median')
# 用转换器对象适配数据
imputer.fit(housing_num)

print(imputer.statistics_)
print(housing_num.median().values)

# 转换数据
X = imputer.transform(housing_num)

print(type(X))
# 也可将其转换回pandas.dataFrame
housing_tr = pd.DataFrame(X, columns = housing_num.columns, index = housing_num.index)
print(type(housing_tr))
print(type(X))

In [ ]:
# Code2.20
# 研究一下文本数据类型（或将其转换为数值型）

from sklearn.preprocessing import OrdinalEncoder

# 从数据集中将文本数据分离出来
housing = strat_train_set.drop('median_house_value', axis = 1)
housing_cat = housing[['ocean_proximity']]

print(housing_cat.head(10))
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
print(housing_cat_encoded[: 10])

print(ordinal_encoder.categories_)

In [ ]:
# Code2.21
# 还是使用独热（one-hot）编码好一些？

from sklearn.preprocessing import OneHotEncoder

# 从数据集中将文本数据分离出来
housing = strat_train_set.drop('median_house_value', axis = 1)
housing_cat = housing[['ocean_proximity']]

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
print(type(housing_cat_1hot))

# 也可以尝试将此稀疏矩阵转换成密集矩阵，但数据量大时最好不要这么做
# print(housing_cat_1hot.toarray())

In [ ]:
# Code2.22
# 自定义转换器

from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

# 实现之前Code2.15功能的自定义转换器
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    #  定义fit方法
    def fit(self, X, y = None):
        return self
    
    #  定义transform方法
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_room = X[:, population_ix] / X[:, households_ix]
        
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_room, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_room]    

In [ ]:
# Code2.23
# 自定义转换器测试

# 从数据集中将文本数据分离出来
housing = strat_train_set.drop('median_house_value', axis = 1)

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room = False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
# Code2.24
# 试试使用流水线工具

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

# 从数据集中去除作为标签的数据
housing = strat_train_set.drop('median_house_value', axis = 1)

# 将数据中的非数字列去掉
housing_num = housing.drop('ocean_proximity', axis = 1)

# 测试一下咱们自己定义的流水线工具
housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
# Code2.25
# 使用ColumnTransformer

from sklearn.compose import ColumnTransformer

# 从数据集中去除作为标签的数据
housing = strat_train_set.drop('median_house_value', axis = 1)

# 将数据中的非数字列去掉
housing_num = housing.drop('ocean_proximity', axis = 1)

num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs),
])

In [ ]:
# Code2.26
# 使用前面分层抽样的结果做训练数据集准备

housing = strat_train_set.drop('median_house_value', axis = 1)
X_train = full_pipeline.fit_transform(housing)
y_train = strat_train_set['median_house_value'].copy()

In [ ]:
# Code2.27
# 终于可以开始训练模型了！先使用LR试试

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

housing_predictions = lin_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
# Code2.28
# 试试更为复杂的模型

from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)

housing_predictions = tree_reg.predict(X_train)
tree_mse = mean_squared_error(y_train, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

In [ ]:
# Code2.29
# 使用k折交叉验证的方法

from sklearn.model_selection import cross_val_score

# sklearn自带的函数很方便，一行代码就搞定
scores = cross_val_score(tree_reg, X_train, y_train, scoring = 'neg_mean_squared_error', cv = 10)
tree_rmse_scores = np.sqrt(-scores)

print('Scores: %s' % tree_rmse_scores)
print('Mean: %s' % tree_rmse_scores.mean())
print('Standard deviation: %s' % tree_rmse_scores.std())

In [ ]:
# Code2.30
# 再用线性模型测试一下交叉验证的结果

scores = cross_val_score(lin_reg, X_train, y_train, scoring = 'neg_mean_squared_error', cv = 10)
lin_rmse_scores = np.sqrt(-scores)

print('Scores: %s' % lin_rmse_scores)
print('Mean: %s' % lin_rmse_scores.mean())
print('Standard deviation: %s' % lin_rmse_scores.std())

In [ ]:
# Code2.31
# 决策树明显过拟合了，使用随机森林的方法试试
# 会运行一分钟左右

from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)

scores = cross_val_score(forest_reg, X_train, y_train, scoring = 'neg_mean_squared_error', cv = 10)
forest_rmse_scores = np.sqrt(-scores)

print('Scores: %s' % forest_rmse_scores)
print('Mean: %s' % forest_rmse_scores.mean())
print('Standard deviation: %s' % forest_rmse_scores.std())

In [ ]:
# Code2.32
# 使用网格搜索自动寻找最优超参数

from sklearn.model_selection import GridSearchCV

# 设置需要测试的超参数值
param_grid = [
    {
        'n_estimators' : [3, 10, 30],
        'max_features': [2, 4, 6, 8],
    },
    {
        'bootstrap': [False],
        'n_estimators': [3, 10],
        'max_features': [2, 3, 4]
    }
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv = 5, scoring = 'neg_mean_squared_error', verbose = 1, n_jobs = -1, return_train_score = True)
grid_search.fit(X_train, y_train)

In [ ]:
# Code 2.33
# 分析结果

# 输出最佳参数
print(grid_search.best_params_)
print(grid_search.best_estimator_)

# 输出分数
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

# 分析每个属性的重要程度
feature_importances = grid_search.best_estimator_.feature_importances_

In [ ]:
# Code 2.33续

# 让输出结果好看些
extra_attribs = ['room_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs

sorted(zip(feature_importances, attributes), reverse = True)

In [ ]:
# Code2.34
# 用测试集评估

# 获取最佳模型
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop('median_house_value', axis = 1)
y_test = strat_test_set['median_house_value'].copy()

X_test = full_pipeline.transform(X_test)

predictions = final_model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
# Code2.35
# 计算一下当前计算误差的95%置信区间

from scipy import stats

confidence = 0.95

squared_errors = (predictions - y_test) ** 2

np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1, loc = squared_errors.mean(), scale = stats.sem(squared_errors)))
